# **Extra 3.1: Clasificación desbalanceada**

<hr>

## **1. Introducción**
En esta práctica adicional se nos pide:

> Crear un clasificador binario que aprenda a predecir si una vuelta ha sido eliminada o no.  

Una vuelta es *"eliminada"* cuando uno de los pilotos comete algún tipo de irregularidad durante la misma, normalmente saltarse los límites de la pista.

Nuestro objetivo es poder detectar este tipo de vueltas utilizando únicamente datos de *velocidades medias* en la *línea de meta* y *máxima en el sector*. 

Estas velocidades normalmente se ven alteradas cuando un piloto no cumple con el reglamento deportivo (exceder límites, no respetar banderas amarillas, no reducir la velocidad del safety car, ir demasiado lento en zonas de salida o de entrada...).

**En el siguiente bloque de código cargamos los datos y recuperamos la columna "Deleted" eliminada durante el preprocesamiento de la práctica 2.**

In [ ]:
import pandas as pd

# Cargar archivos de las practicas 2.2 y 3.1
data_full = pd.read_csv("https://raw.githubusercontent.com/AIC-Uniovi/Sistemas-Inteligentes/refs/heads/main/datasets/f1_23_monaco.csv")
data_reduced = pd.read_pickle("https://raw.githubusercontent.com/AIC-Uniovi/Sistemas-Inteligentes/refs/heads/main/datasets/f1_23_monaco.pkl")

#Fusionar columnas faltantes sin las filas redundantes
data_full["Time"] = data_full["Time"].astype(str)
data_reduced["Time"] = data_reduced["Time"].astype(str)
cols_to_add = [col for col in data_full.columns if col not in data_reduced.columns]
data = data_reduced.merge( data_full[["Time"] + cols_to_add], on="Time", how="left" )
data = data.dropna(subset=["SpeedI1"])

## **2. Preprocesamiento y visualización**

<div class="alert alert-block alert-info">
    <b>Ejercicio:</b> Crea un nuevo DataFrame <code>data_vel</code> con las columnas relevantes para nuestro problema ( <i>"SpeedI1", "SpeedI2", "SpeedFL", "SpeedST" y "Deleted"<i>). Por simplificar, nos quedamos solo con los equipos <b>Williams</b> y <b>Alpine</b>.
</div>

In [ ]:
equipos = ["Williams", "Alpine"]
data_vel = data.loc[data["Team"].isin(equipos), ["Team", "SpeedI1", "SpeedI2", "SpeedFL", "SpeedST", "Deleted"]].copy()

<div class="alert alert-block alert-info">
    <b>Ejercicio:</b> Añade la columna "Class" al DataFrame.
</div>

In [ ]:
data_vel["Class"] = data_vel["Deleted"].apply(lambda x: 1 if x == True else 0)

<div class="alert alert-block alert-info">
    <b>Ejercicio:</b> Visualiza la distribución de velocidad máxima en el sector ("SpeedST") para ambos equipos.
</div>

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

# Con un KDE
plt.figure(figsize=(10,4))
sns.kdeplot(data=data_vel, x="SpeedST", hue="Team", fill=True)
plt.title("Distribución por equipo")
plt.show()

<div class="alert alert-block alert-info">
    <b>Ejercicio:</b> Separa el conjunto en <code>data_vel_train</code> y <code>data_vel_test</code> utilizando la función <code>train_test_split</code>. Utiliza el 30% para test.
</div>

In [ ]:
from sklearn.model_selection import train_test_split
seed = 2533
data_vel_train, data_vel_test = train_test_split(data_vel, test_size=0.30, random_state=seed)

## **3. Baselines**

<div class="alert alert-block alert-info">
    <b>Ejercicio:</b> Entrena los baselines <i>Aleatorio</i> y <i>Zero-R</i>. Recuerda separar previamente la X e Y de entrenamiento.
</div>

In [ ]:
from sklearn.dummy import DummyClassifier

X_train = data_vel_train[["SpeedI1", "SpeedI2", "SpeedFL", "SpeedST"]]
Y_train = data_vel_train["Class"]

baseline_aleatorio = DummyClassifier(strategy="uniform", random_state=seed)
baseline_zeror = DummyClassifier(strategy="most_frequent")

baseline_aleatorio.fit(X_train, Y_train)
baseline_zeror.fit(X_train, Y_train)

<div class="alert alert-block alert-info">
    <b>Ejercicio:</b> Separa la X e Y de test y realiza una predicción con cada modelo (<code>pred_aleatorio</code> y <code>pred_zeror</code>) .
</div>

In [ ]:
X_test = data_vel_test[["SpeedI1", "SpeedI2", "SpeedFL", "SpeedST"]]
Y_test = data_vel_test["Class"]

pred_aleatorio = baseline_aleatorio.predict(X_test)
pred_zeror = baseline_zeror.predict(X_test)

Ya tenemos nuestros modelos entrenados y hemos realizado predicciones sobre el conjunto de test, así que ahora pasamos a evaluar su rendimiento de forma un poco más objetiva.

Para ello utilizaremos de nuevo el módulo **"metrics"** de la librería **"scikit-learn"**.

Recueda que las métricas más relevantes en problemas de clasificación son las siguientes:

<div style="width:800px;background:white;padding:10px">
    <img src="https://i.imgur.com/7WwY9bZ.jpeg" style="margin-bottom:10px"> </img>
</div>

Para nuestro problema concreto significarían lo siguiente:

* **Accuracy:** Porcentaje de aciertos del modelo sobre el total de predicciones. Útil cuando las clases están balanceadas.
* **Precision:** De todas las veces que el modelo predijo un 1 (deleted), ¿Cuántas veces acertó?
* **Recall:** De todas las vueltas que fueron realmente eliminadas, ¿Cuántas fue capaz de detectar el modelo?
* **F1-Score:** Media "armónica" entre Precision y Recall.

<div class="alert alert-block alert-info">
    <b>Ejercicio:</b> Utiliza las funciones de <code>sklearn.metrics</code> para evaluar los modelos <i>Aleatorio</i> y <i>Zero-R</i>. Obtén métricas <i>Accuracy</i>, <i>Precision</i>, <i>Recall</i> y <i>F1-score</i> así como la <i>Matriz de confusión</i>.
</div>

In [ ]:
from sklearn import metrics

print("-"*50)
print("Aleatorio")
print("-"*50)
print("· Confusion matrix:")
print(metrics.confusion_matrix(Y_test, pred_aleatorio))
print("· Accuracy:", metrics.accuracy_score(Y_test, pred_aleatorio))
print("· Precision:", metrics.precision_score(Y_test, pred_aleatorio))
print("· Recall:", metrics.recall_score(Y_test, pred_aleatorio))
print("· F1 Score:", metrics.f1_score(Y_test, pred_aleatorio))
print()
print("-"*50)
print("Zero-R")
print("-"*50)
print("· Confusion matrix:")
print(metrics.confusion_matrix(Y_test, pred_zeror))
print("· Accuracy:", metrics.accuracy_score(Y_test, pred_zeror))
print("· Precision:", metrics.precision_score(Y_test, pred_zeror))
print("· Recall:", metrics.recall_score(Y_test, pred_zeror))
print("· F1 Score:", metrics.f1_score(Y_test, pred_zeror))


Los resultados obtenidos, salvo en la Accuracy, dejan mucho que desear.

Como recordarás, la Accuracy no es una buena métrica cuando nos encontramos ante un conjunto con **desbalanceo de clases**.

<div class="alert alert-block alert-info">
    <b>Ejercicio:</b> Obtén el número de ejemplos de cada clase (positivo y negativo) en el conjunto <code>data_vel</code> sin dividir.
</div>

In [ ]:
data_vel.groupby("Class").size()

Este desbalance se produce porque hay muchas más vueltas "normales" (clase $0$) que vueltas "eliminadas" (clase $1$).
Por tanto no podemos fijarnos en la accuracy y tendremos que aprender algún modelo diferente que maximice:

- El **Recall**, para evitar los falsos negativos (vueltas eliminadas que el modelo no detectó).
- La **Precision**, para evitar los falsos positivos (vueltas normales que el modelo clasifica como eliminadas.).
- El **F1-score**, si nos interesa un equilibrio entre las dos anteriores.

Probaremos a entrenar ahora modelos más complejos a fin de mejorar estas métricas.

## **4. Aprendizaje**

Los siguientes modelos que vamos a aprender, a diferencia de los baselines, sí utilizan los datos de entrada ($X$) para predecir las salidas ($Y$); por tanto, a partir de ahora es crucial estandarizar estos datos.

Recuerda que esto nos sirve para evitar que las variables que se miden en escalas más grandes dominen a las que se miden en escalas más pequeñas, lo que ralentizaría el aprendizaje de los modelos.

<div class="alert alert-block alert-info">
    <b>Ejercicio:</b> Utiliza el <a href="https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.StandardScaler.html"><code>StandardScaler()</code></a> de <i>sklearn</i> para estandarizar las X de train y de test. Almacena los nuevos datos en <code>X_train_std</code> y <code>X_test_std</code>. 
    <hr>
    Recuerda que los datos de test son desconocidos para nosotros, por lo que no pueden influir a la hora de calcular la media y las desviación. 
    <hr>
    Cómo utilizar el <code>StandardScaler()</code>:
    <ul>
        <li> <code>fit_transform(X_train)</code>: Solo debe hacerse con los datos de entrenamiento. Este método hace dos pasos: obtiene las medias y desviaciones de cada columna (<code>fit()</code>) y estandariza en base a ellas (<code>transform()</code>) .</li>
        <li> <code>transform(X_test)</code>: Utiliza las medias y desviaciones calculadas en el conjunto de train (con el <code>fit()</code>) para estandarizar.</li>
    </ul>

</div>

In [ ]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
X_train_std = scaler.fit_transform(X_train)
X_test_std = scaler.transform(X_test)

print(X_train_std.mean(axis=0))
print(X_train_std.std(axis=0))

print(X_test_std.mean(axis=0))
print(X_test_std.std(axis=0))


<div class="alert alert-block alert-info">
    <b>Ejercicio:</b> Entrena diversos modelos aprovechando las funciones definidas previamente en la práctica 3.1.
</div>


In [ ]:
def train_and_eval_model(model_name, model, X_train_std, Y_train, X_test_std, Y_test):
    # Entrenar el modelo
    model.fit(X_train_std, Y_train.squeeze())
    # Predicciones
    Y_train_pred = model.predict(X_train_std)
    Y_test_pred = model.predict(X_test_std)
    # Calcular métricas para train
    tr_accuracy = metrics.accuracy_score(Y_train, Y_train_pred)
    tr_precision = metrics.precision_score(Y_train, Y_train_pred, zero_division=0)
    tr_recall = metrics.recall_score(Y_train, Y_train_pred)
    tr_f1 = metrics.f1_score(Y_train, Y_train_pred)
    
    # Calcular métricas para test
    tst_accuracy = metrics.accuracy_score(Y_test, Y_test_pred)
    tst_precision = metrics.precision_score(Y_test, Y_test_pred, zero_division=0)
    tst_recall = metrics.recall_score(Y_test, Y_test_pred)
    tst_f1 = metrics.f1_score(Y_test, Y_test_pred)
    return (model_name, tr_accuracy, tr_precision, tr_recall, tr_f1, tst_accuracy, tst_precision, tst_recall, tst_f1)

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.svm import SVC


def train_and_eval(X_train_std, Y_train, X_test_std, Y_test):
    # Creamos una lista donde almacenar los resultados de cada modelo
    all_results = []
    
    # Baseline aleatorio
    the_model = DummyClassifier(strategy="uniform", random_state=seed)
    model_results = train_and_eval_model("Aleatorio", the_model, X_train_std, Y_train, X_test_std, Y_test)
    all_results.append(model_results)
    
    # Baseline Zero-R
    the_model = DummyClassifier(strategy="most_frequent")
    model_results = train_and_eval_model("Zero-R", the_model, X_train_std, Y_train, X_test_std, Y_test)
    all_results.append(model_results)
    
    # Baseline Log. Reg
    the_model = LogisticRegression()
    model_results = train_and_eval_model("Log. Reg.", the_model, X_train_std, Y_train, X_test_std, Y_test)
    all_results.append(model_results)
    
    # Baseline KNN
    the_model = KNeighborsClassifier(n_neighbors=3)
    model_results = train_and_eval_model("KNN", the_model, X_train_std, Y_train, X_test_std, Y_test)
    all_results.append(model_results)
    
    # Baseline Tree
    the_model =DecisionTreeClassifier(random_state=seed, max_depth=2)
    model_results = train_and_eval_model("Tree", the_model, X_train_std, Y_train, X_test_std, Y_test)
    all_results.append(model_results)
    
    # Baseline SVM Lineal
    the_model =  SVC(kernel='linear')
    model_results = train_and_eval_model("SVM Lineal", the_model, X_train_std, Y_train, X_test_std, Y_test)
    all_results.append(model_results)
    
    # Baseline SVM polinómico
    the_model =  SVC(kernel='poly', degree=2, coef0=1)
    model_results = train_and_eval_model("SVM poli.", the_model, X_train_std, Y_train, X_test_std, Y_test)
    all_results.append(model_results)
    
    
    # Imprimir el dataframe resultante
    multi_index = pd.MultiIndex.from_tuples([ ("Modelo", "Nombre"), ("Train", "Accuracy"), ("Train", "Precision"), ("Train", "Recall"), ("Train", "F1"), ("Test", "Accuracy"), ("Test", "Precision"), ("Test", "Recall"), ("Test", "F1") ])    
    all_results = pd.DataFrame(all_results, columns=multi_index)
    display(all_results)

train_and_eval(X_train_std, Y_train, X_test_std, Y_test)

## **5. Análisis de resultados**

**¿Cuáles son las razones de su bajo rendimiento?**

Con un desbalance de clases de esta magnitud, los modelos **aprenden a predecir solo la clase mayoritaria (0)**, generando altos valores de Accuracy pero bajos de Precision, Recall y F1. 

Esto explica el rendimiento pobre en métricas relevantes para la clase positiva, a pesar de una Accuracy aceptable.

El árbol de decisión destaca un poco, intentando detectar positivos en entrenamiento, aunque sin éxito en el test.

Ningún otro modelo detecta la clase positiva correctamente. Regresión Logística, KNN o SVM tienen recall y F1-score nulos, prediciendo solo la clase mayoritaria.

Otro factor que podría estar afectando al rendimiento es la insuficiencia de los datos en la entrada ($X$). Puede que necesitemos añadir alguna columna más de nuestro dataset como entrada para mejorar el rendimiento.

<div class="alert alert-block alert-info">
    <b>Ejercicio:</b> Modifica algún hiperparámetro del mejor modelo buscando mejorar su resultado.
</div>


In [ ]:
def train_and_eval(X_train_std, Y, X_test_std, Y_test):
    # Creamos una lista donde almacenar los resultados de cada modelo
    all_results = []

    # Baseline Tree (le ponemos class_weight balanced, podemos probar con más depth)
    the_model =DecisionTreeClassifier(random_state=seed, max_depth=3, class_weight='balanced', min_samples_split=4,  min_samples_leaf=2)
    model_results = train_and_eval_model("Tree", the_model, X_train_std, Y, X_test_std, Y_test)
    all_results.append(model_results)
    
    # Imprimir el dataframe resultante
    multi_index = pd.MultiIndex.from_tuples([ ("Modelo", "Nombre"), ("Train", "Accuracy"), ("Train", "Precision"), ("Train", "Recall"), ("Train", "F1"), ("Test", "Accuracy"), ("Test", "Precision"), ("Test", "Recall"), ("Test", "F1") ])    
    all_results = pd.DataFrame(all_results, columns=multi_index)
    display(all_results)

train_and_eval(X_train_std, Y_train, X_test_std, Y_test)

Como se puede observar, incluso al variar con los hiperparámetros, la diferencia entre las clases es tan exagerada que hace imposible que el modelo tenga un buen rendimiento.

Una posible solución en este caso sería la ya mencionada inclusión de alguna columna adicional (puede que los tiempos por vuelta o sector nos ayuden) o simplemente realizar alguna estrategia de **oversampling**.

<div class="alert alert-block alert-warning">
    <strong>Oversampling:</strong> Técnica para equilibrar el número de ejemplos de cada clase mediante la repetición de las muestras minoritarias.
</div>
